# Landgrid Geoprocessing
Example of using Pythonic libraries Fiona, Shapely, geopandas, pyproj, and pyscopg2 to load and process an ESRI File Geodatabase.

## Imports and Globals

In [1]:
!ls $DIML_HOME/database

indexes.sql  schema.sql


In [2]:
import os
from pprint import pprint
from collections import namedtuple

import fiona
import geojson
import geopandas as gpd
import psycopg2
from shapely.geometry import shape, Polygon, MultiPolygon, box, mapping

twp = namedtuple('twp', 'layer state') 

In [5]:
# Postgis
pg_host = os.environ["PG_SERVER"]
pg_db = os.environ["PG_DATABASE"]
pg_user = os.environ["PG_UID"]
pg_pwd = os.environ["PG_PWD"]

# WGS84 (epsg:4326)
gdb_path = os.path.join(os.environ["DATA_DIR"], 'landgrid', 'DI_basemaps_WGS84.gdb')

layers = ['States_US', 'Counties_US_WGS84', 'OK_section', 'OK_township']

## Create Spatial Schema

In [8]:
ddl_path = os.path.join(os.environ["DIML_HOME"], 'database', 'schema.sql')

with psycopg2.connect(host=pg_host, database=pg_db,
                      user=pg_user, password=pg_pwd) as c, \
        open(ddl_path, 'r') as f:

    try:
        # Execute DDL
        cursor = c.cursor()
        cursor.execute(f.read())
        
        c.commit()
        cursor.close()
        
    except Exception as e:
        c.rollback()
        raise e

print("Completed.")

Completed.


## Load Spatial Data as WGS83 (epsg:4326) Geometry
- Reference [here](https://github.com/Toblerity/Fiona/blob/2ec38d087fea72c8bd0e7696d8ac1a6203df8851/examples/with-shapely.py#L22)
- Also, creates a bounding box and loads as geoJSON.

### Insert States and Tesselated States

In [10]:
%%time

dml = """
INSERT INTO landgrid.states_us (State_Name, Shape_Length, 
                                Shape_Area, geobounds, shape)
VALUES (%s, %s, %s, %s, ST_GeomFromText(%s, 4326));
"""

dml2 = """
INSERT INTO landgrid.states_us_grid (State_Name, shape)
SELECT 	State_Name, ST_Subdivide(shape, 255) AS grid_shape
FROM	landgrid.states_us
;
"""

with psycopg2.connect(host=pg_host, database=pg_db, 
                      user=pg_user, password=pg_pwd) as db, \
        fiona.open(gdb_path, layer='States_US') as src:
    try:
        pprint(f"Schema: {src.schema}")
        
        cursor = db.cursor()

        print(f"Starting load of {len(src)} states...")

        for rec in src:
            
            poly = shape(rec['geometry'])
            if not poly.is_valid:
                print(f"Cleaning {rec['properties']['State_Name']}...")
                clean = poly.buffer(0.0)
                assert clean.is_valid
                assert clean.geom_type == 'MultiPolygon'
                poly = clean
            
            bbox = box(*poly.bounds)
            g_json = geojson.dumps(mapping(bbox), sort_keys=True)
            
            cursor.execute(dml, (rec['properties']['State_Name'], 
                                 rec['properties']['Shape_Length'], 
                                 rec['properties']['Shape_Area'],
                                 g_json,
                                 poly.wkt))
            
            print(f"Processed {rec['id']} of {len(src)}: {rec['properties']['State_Name']}")
            

        db.commit()
            
        print("Completed state load.")
        
        print("Starting tesselated state load...")
        
        cursor.execute(dml2)
        db.commit()
        cursor.close()
        
        print("Completed tesselated state load.")
        
        # TODO: Rebuild stats
        #VACUUM ANALYZE [table_name] [(column_name)];
        
    except Exception as e:
        db.rollback()
        raise e

("Schema: {'properties': OrderedDict([('State_Name', 'str:50'), "
 "('Shape_Length', 'float'), ('Shape_Area', 'float')]), 'geometry': "
 "'MultiPolygon'}")
Starting load of 51 states...
Processed 2 of 51: Alabama
Cleaning Alaska...
Processed 3 of 51: Alaska
Processed 4 of 51: Arizona
Processed 5 of 51: Arkansas
Processed 6 of 51: California
Processed 7 of 51: Colorado
Cleaning Connecticut...
Processed 8 of 51: Connecticut
Processed 9 of 51: Delaware
Processed 10 of 51: District of Columbia
Processed 11 of 51: Florida
Processed 12 of 51: Georgia
Processed 13 of 51: Hawaii
Processed 14 of 51: Idaho
Processed 15 of 51: Illinois
Processed 16 of 51: Indiana
Processed 17 of 51: Iowa
Processed 18 of 51: Kansas
Processed 19 of 51: Kentucky
Cleaning Louisiana...
Processed 20 of 51: Louisiana
Processed 21 of 51: Maine
Processed 22 of 51: Maryland
Processed 23 of 51: Massachusetts
Cleaning Michigan...
Processed 24 of 51: Michigan
Processed 25 of 51: Minnesota
Processed 26 of 51: Mississippi
Proce

## Insert Counties

In [11]:
%%time

dml = """
INSERT INTO landgrid.counties_us (County_Name, State_Name, CountyID, 
                                  StateID, FIPS_State, FIPS_County, 
                                  API_State, API_County, LAT, LON,
                                  Shape_Length, Shape_Area, geobounds, 
                                  shape)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
        ST_GeomFromText(%s, 4326));
"""

with psycopg2.connect(host=pg_host, database=pg_db, 
                      user=pg_user, password=pg_pwd) as db, \
        fiona.open(gdb_path, layer='Counties_US_WGS84') as src:
    try:
        pprint(f"Schema: {src.schema}")
        
        cursor = db.cursor()
        
        total = len(src)
        print(f"Starting load of {total} counties...")

        for i, rec in enumerate(src):
            
            poly = shape(rec['geometry'])
            if not poly.is_valid:
                print(f"Cleaning {rec['properties']['County_Name']}...")
                clean = poly.buffer(0.0)
                assert clean.is_valid, f"Invalid County {rec['properties']['County_Name']}!"
                assert clean.geom_type == 'MultiPolygon' or clean.geom_type == 'Polygon', \
                       f'{clean.geom_type} is not a Polygon!'
                poly = clean
            
            bbox = box(*poly.bounds)
            g_json = geojson.dumps(mapping(bbox), sort_keys=True)
            
            cursor.execute(dml, (rec['properties']['County_Name'], 
                                 rec['properties']['State_Name'],
                                 rec['properties']['CountyID'],
                                 rec['properties']['StateID'],
                                 rec['properties']['FIPS_State'],
                                 rec['properties']['FIPS_County'],
                                 rec['properties']['API_State'],
                                 rec['properties']['API_County'],
                                 rec['properties']['LAT'],
                                 rec['properties']['LON'],
                                 rec['properties']['Shape_Length'], 
                                 rec['properties']['Shape_Area'],
                                 g_json,
                                 poly.wkt))
            if i % 500 == 0:
                print(f"{round(i/total*100, 2)}%: {i} of {total}: {rec['properties']['County_Name']}")

        db.commit()
        cursor.close()

        print("Completed county load.")
        
    except Exception as e:
        db.rollback()
        raise e

("Schema: {'properties': OrderedDict([('County_Name', 'str:50'), "
 "('State_Name', 'str:50'), ('CountyID', 'int'), ('StateID', 'int'), "
 "('FIPS_State', 'str:2'), ('FIPS_County', 'str:3'), ('API_State', 'str:2'), "
 "('API_County', 'str:3'), ('LAT', 'float'), ('LON', 'float'), "
 "('Shape_Length', 'float'), ('Shape_Area', 'float')]), 'geometry': "
 "'MultiPolygon'}")
Starting load of 3138 counties...
0.0%: 0 of 3138: Autauga
Cleaning Marshall...
Cleaning Pike...
Cleaning Schley...
Cleaning Terrell...
15.93%: 500 of 3138: Parke
Cleaning Plaquemines...
31.87%: 1000 of 3138: Belknap
Cleaning Roane...
47.8%: 1500 of 3138: Orange
Cleaning Washington...
Cleaning Williamsburg...
Cleaning Lake and Peninsula...
Cleaning North Slope...
63.73%: 2000 of 3138: Mills
79.67%: 2500 of 3138: Cass
95.6%: 3000 of 3138: Milam
Cleaning Caddo...
Completed county load.
CPU times: user 1min 17s, sys: 527 ms, total: 1min 18s
Wall time: 1min 37s


### Insert Townships and Clean Polygons

In [12]:
%%time

dml = """
INSERT INTO landgrid.plss_township (TWPCODE, MER, MST, TWP, THALF, 
                                    TNS, RGE, RHALF, REW, SecCount, 
                                    TWPLabel, Shape_Length, Shape_Area, 
                                    State_Name, geobounds, shape)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
        ST_GeomFromText(%s, 4326));
"""

try:
    with psycopg2.connect(host=pg_host, database=pg_db,user=pg_user, password=pg_pwd) as db:

        cursor = db.cursor()
        cursor.execute('TRUNCATE TABLE landgrid.plss_township;');  # restart
        
        twp_queue = [twp('OK_township', 'Oklahoma'), twp('NM_township', 'New Mexico')]

        for ts in twp_queue:
            with fiona.open(gdb_path, layer=ts.layer) as src:

                total = len(src)
                print(f"Starting load of {ts.layer}: {total} townships...")
                pprint(f"Schema: {src.schema}")

                for i, rec in enumerate(src):

                    poly = shape(rec['geometry'])
                    if not poly.is_valid:
                        print(f"Cleaning {rec['properties']['TWPLabel']}...")

                        clean = poly.buffer(0.0)
                        assert clean.is_valid, 'Invalid Polygon!'
                        assert clean.geom_type == 'MultiPolygon' or clean.geom_type == 'Polygon', \
                                f'{clean.geom_type} is not a Polygon!'
                        poly = clean

                    bbox = box(*poly.bounds)
                    g_json = geojson.dumps(mapping(bbox), sort_keys=True)

                    cursor.execute(dml, (rec['properties']['TWPCODE'], 
                                         rec['properties']['MER'], 
                                         rec['properties']['MST'],
                                         rec['properties']['TWP'],
                                         rec['properties']['THALF'],
                                         rec['properties']['TNS'],
                                         rec['properties']['RGE'],
                                         rec['properties']['RHALF'],
                                         rec['properties']['REW'],
                                         rec['properties']['SecCount'],
                                         rec['properties']['TWPLabel'],
                                         rec['properties']['Shape_Length'],
                                         rec['properties']['Shape_Area'],
                                         ts.state,  # TMP
                                         g_json,
                                         poly.wkt))
                    if i % 500 == 0:
                        print(f"{round(i/total*100, 2)}%: {i} of {total}: {rec['properties']['TWPLabel']}")

                db.commit()

                print(f"Completed {ts.layer} load.")
        
        cursor.close()

except Exception as e:
    db.rollback()
    raise e

Starting load of OK_township: 2075 townships...
("Schema: {'properties': OrderedDict([('TWPCODE', 'str:24'), ('MER', 'str:5'), "
 "('MST', 'str:20'), ('TWP', 'int'), ('THALF', 'int'), ('TNS', 'str:1'), "
 "('RGE', 'int'), ('RHALF', 'int'), ('REW', 'str:1'), ('SecCount', 'int'), "
 "('TWPLabel', 'str:20'), ('Shape_Length', 'float'), ('Shape_Area', "
 "'float')]), 'geometry': 'MultiPolygon'}")
0.0%: 0 of 2075: T1N R1E
24.1%: 500 of 2075: T3S R23E
48.19%: 1000 of 2075: T9N R19W
Cleaning T13N R22W...
72.29%: 1500 of 2075: T18N R21E
96.39%: 2000 of 2075: T28N R14W
Completed OK_township load.
Starting load of NM_township: 3525 townships...
("Schema: {'properties': OrderedDict([('TWPCODE', 'str:24'), ('MER', 'str:5'), "
 "('MST', 'str:20'), ('TWP', 'int'), ('THALF', 'int'), ('TNS', 'str:1'), "
 "('RGE', 'int'), ('RHALF', 'int'), ('REW', 'str:1'), ('SecCount', 'int'), "
 "('TWPLabel', 'str:20'), ('Shape_Length', 'float'), ('Shape_Area', "
 "'float')]), 'geometry': 'MultiPolygon'}")
0.0%: 0 of 

## Calculate State Overlaps with ST_Intersects Using Tesselated State Table
- This is run once for all PLSS states

In [13]:
%%time

dml = """
WITH ov_st
AS
(
	SELECT	ov.TownshipId, STRING_AGG(ov.State_Name, ', ') AS State_Overlaps
	FROM	(
		SELECT 	DISTINCT twp.TownshipId, st.State_Name
		FROM 	landgrid.plss_township AS twp 
			INNER JOIN landgrid.states_us_grid AS st
				ON ST_Intersects(st.shape, twp.shape)
        --ORDER BY twp.TownshipId, st.State_Name
	) AS ov
	GROUP BY ov.TownshipId
)
UPDATE 	landgrid.plss_township
	SET State_Overlaps = ov.State_Overlaps
FROM	ov_st AS ov
WHERE 	landgrid.plss_township.TownshipId = ov.TownshipId
"""

with psycopg2.connect(host=pg_host, database=pg_db,user=pg_user, password=pg_pwd) as db:

    try:
        # Execute dml
        cursor = db.cursor()
        cursor.execute(dml)
        db.commit()
        cursor.close()

    except Exception as e:
        db.rollback()
        raise e

print("Completed.")

Completed.
CPU times: user 3.48 ms, sys: 32 µs, total: 3.51 ms
Wall time: 687 ms


## Calculate County Overlaps Using GPD
- TWPCODE can be used as a unique identifier
- Some townships are over water and have no county intersection (e.g. 1718T02200SR01900E)
- Some counties across states are named the same and need to be de-duped
- Native database spatial join seems more efficient.

In [14]:
%%time

# In WGS84: EPSG:4326
layers = ['Counties_US_WGS84', 'LA_township']
gdb_path = os.path.join(os.environ["DATA_DIR"], 'landgrid', 'DI_basemaps_WGS84.gdb')

counties_df = gpd.read_file(gdb_path, layer='Counties_US_WGS84')
la_twp_df = gpd.read_file(gdb_path, layer='LA_township')  # 1469

# pre-generate sindex if it doesn't already exist
la_twp_df.sindex

ovlaps_df = gpd.sjoin(la_twp_df, counties_df, how='left', op='intersects')

# Handle null join
ovlaps_pre_df = ovlaps_df.loc[:, ['TWPCODE', 'County_Name']].copy()
ovlaps_pre_df.loc[ovlaps_pre_df.County_Name.isna(), ['County_Name']] = 'Unknown'

ovlaps_join_df = (ovlaps_pre_df
                  .drop_duplicates()  # handle dup county names
                  .groupby('TWPCODE', sort=False)
                  .aggregate(County_Overlaps=('County_Name', ','.join))  # Pandas 0.25.0
                  .reset_index()
                 )

la_final_df = la_twp_df.merge(ovlaps_join_df, how='inner', on='TWPCODE')

CPU times: user 7.23 s, sys: 168 ms, total: 7.4 s
Wall time: 7.39 s


### Write to DB

In [15]:
%%time

ddl = """
DELETE FROM landgrid.plss_township 
WHERE State_Name = %s;
"""

dml = """
INSERT INTO landgrid.plss_township (TWPCODE, MER, MST, TWP, THALF, 
                                   TNS, RGE, RHALF, REW, SecCount, 
                                   TWPLabel, Shape_Length, Shape_Area, 
                                   State_Name, County_Overlaps, geobounds, 
                                   shape)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
        ST_GeomFromText(%s, 4326));
"""

try:
    with psycopg2.connect(host=pg_host, database=pg_db,user=pg_user, password=pg_pwd) as db:
        
        twp_queue = [twp('LA_township', 'Louisiana')]

        cursor = db.cursor()

        total = la_final_df.shape[0]
        print(f"Starting load of {ts.state}: {total} townships...")

        for ts in twp_queue:

            cursor.execute(ddl, [ts.state]);  # restart
            
            for rec in la_final_df.itertuples():

                poly = shape(rec.geometry)
                if not poly.is_valid:
                    print(f"Cleaning {rec.Index}: {rec.TWPLabel}...")

                    clean = poly.buffer(0.0)
                    assert clean.is_valid, 'Invalid Polygon!'
                    assert clean.geom_type == 'MultiPolygon' or clean.geom_type == 'Polygon', \
                            f'{clean.geom_type} is not a Polygon!'
                    poly = clean

                bbox = box(*poly.bounds)
                g_json = geojson.dumps(mapping(bbox), sort_keys=True)
                
                cursor.execute(dml, (rec.TWPCODE, 
                                     rec.MER, 
                                     rec.MST,
                                     rec.TWP,
                                     rec.THALF,
                                     rec.TNS,
                                     rec.RGE,
                                     rec.RHALF,
                                     rec.REW,
                                     rec.SecCount,
                                     rec.TWPLabel,
                                     rec.Shape_Length,
                                     rec.Shape_Area,
                                     ts.state,  # TMP
                                     rec.County_Overlaps,                                         
                                     g_json,
                                     poly.wkt))

                if rec.Index % 500 == 0:
                    print(f"{round(rec.Index/total*100, 2)}%: {rec.Index} of {total}: {rec.TWPLabel}")

            db.commit()

            print(f"Completed {ts.layer} load.")
            
        cursor.close()

except Exception as e:
    db.rollback()
    raise e

Starting load of New Mexico: 1469 townships...
0.0%: 0 of 1469: T1N R1E
Cleaning 167: T4S R11E...
Cleaning 457: T10S R11E...
34.04%: 500 of 1469: T11S R4E
Cleaning 687: T14S R3E...
Cleaning 858: T17N R2E...
Cleaning 959: T18S R22E...
Cleaning 1000: T19S R18E...
68.07%: 1000 of 1469: T19S R18E
Cleaning 1177: T23N R12E...
Cleaning 1191: T23S R22E...
Cleaning 1358: T9S R14E...
Cleaning 1406: T12S R18E...
Completed LA_township load.
CPU times: user 1.69 s, sys: 72.5 ms, total: 1.77 s
Wall time: 2.33 s


## Dissolve to New Feature Class Using GPD
**TODO**: Consider cleaning polygons before the dissolve.

- All in WGS84: EPSG:4326
- For Texas_Abstracts, dissolve (Township, Block) => Block
- For Ohio_Sections, dissolve (COUNTY, TOWNSHIP) => Municipality
- For Ohio_Sections, dissolve (COUNTY, TOWNSHIP, TWP, TNS, RGE, REW) => Township
- For Ohio_Sections, dissolve (COUNTY, TOWNSHIP, TWP, TNS, RGE, REW, SEC) => Section

In [16]:
%%time

dml = """
INSERT INTO landgrid.ohio_municipality (COUNTY, TOWNSHIP, State_Name, geobounds, 
                                        shape)
VALUES (%s, %s, %s, %s, ST_GeomFromText(%s, 4326));
"""

print(f"Starting shapefile load...")

gdb_path = os.path.join(os.environ["DATA_DIR"], 'landgrid', 'DI_basemaps_WGS84.gdb')
ohio_df = gpd.read_file(gdb_path, layer='Ohio_Sections')

print(f"Finished shapefile load.")

try:
    ts = twp('Ohio_Sections', 'Ohio')
    
    print(f"Starting {ts.state} dissolve...")
    
    ohio_muni_df = (ohio_df.loc[:, ['COUNTY', 'TOWNSHIP', 'geometry']].copy()
                    .dissolve(by=['COUNTY', 'TOWNSHIP'], aggfunc='first')
                    .reset_index()
                   )
    
    print(f"Finished {ts.state} dissolve")

    with psycopg2.connect(host=pg_host, database=pg_db,user=pg_user, password=pg_pwd) as db:
        
        twp_queue = [twp('Ohio_Sections', 'Ohio')]

        cursor = db.cursor()

        total = ohio_muni_df.shape[0]
        print(f"Starting load of {ts.state}: {total} townships...")

        for ts in twp_queue:

            cursor.execute('TRUNCATE TABLE landgrid.ohio_municipality;');  # restart
            
            for rec in ohio_muni_df.itertuples():

                poly = shape(rec.geometry)
                if not poly.is_valid:
                    print(f"Cleaning {rec.Index}: {rec.COUNTY}, {rec.TOWNSHIP}...")

                    clean = poly.buffer(0.0)
                    assert clean.is_valid, 'Invalid Polygon!'
                    assert clean.geom_type == 'MultiPolygon' or clean.geom_type == 'Polygon', \
                            f'{clean.geom_type} is not a Polygon!'
                    poly = clean

                bbox = box(*poly.bounds)
                g_json = geojson.dumps(mapping(bbox), sort_keys=True)
                
                cursor.execute(dml, (rec.COUNTY, 
                                     rec.TOWNSHIP, 
                                     ts.state,  # TMP
                                     #rec.County_Overlaps,  # TODO
                                     g_json,
                                     poly.wkt))

                if rec.Index % 500 == 0:
                    print(f"{round(rec.Index/total*100, 2)}%: {rec.Index} of {total}: {rec.TOWNSHIP}")

            db.commit()

            print(f"Completed {ts.layer} load.")

        cursor.close()

except Exception as e:
    db.rollback()
    raise e

Starting shapefile load...
Finished shapefile load.
Starting Louisiana dissolve...
Finished Louisiana dissolve
Starting load of Louisiana: 1363 townships...
0.0%: 0 of 1363: BRATTON
Cleaning 134: Butler, MADISON...
36.68%: 500 of 1363: PLEASANT
Cleaning 730: Lucas, JERUSALEM...
Cleaning 910: Muskingum, HARRISON...
73.37%: 1000 of 1363: JACKSON
Completed Ohio_Sections load.
CPU times: user 24.5 s, sys: 152 ms, total: 24.6 s
Wall time: 25.5 s


### Load Ohio Sections

In [17]:
%%time

dml = """
INSERT INTO landgrid.ohio_sections (SUBDIV_NM, TWP, TNS, RGE, REW, SEC, 
                                    QTR_TWP, ALLOTMENT, TRACT, LOT, DIVISION, 
                                    FRACTION, COUNTY, TOWNSHIP, SURVEY_TYP, 
                                    ObjectID, VMSLOT, OTHER_SUB, Shape_Length, 
                                    Shape_Area, State_Name, geobounds, shape)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
        %s, %s, %s, %s, %s, ST_GeomFromText(%s, 4326));
"""

try:
    with psycopg2.connect(host=pg_host, database=pg_db,user=pg_user, password=pg_pwd) as db:

        cursor = db.cursor()
        cursor.execute('TRUNCATE TABLE landgrid.ohio_sections;');  # restart
        
        twp_queue = [twp('Ohio_Sections', 'Ohio')]

        for ts in twp_queue:
            with fiona.open(gdb_path, layer=ts.layer) as src:

                total = len(src)
                print(f"Starting load of {ts.layer}: {total} sections...")
                pprint(f"Schema: {src.schema}")

                for i, rec in enumerate(src):

                    poly = shape(rec['geometry'])
                    if not poly.is_valid:
                        print(f"Cleaning {rec['properties']['TOWNSHIP']}...")

                        clean = poly.buffer(0.0)
                        assert clean.is_valid, 'Invalid Polygon!'
                        assert clean.geom_type == 'Polygon' or clean.geom_type == 'MultiPolygon', \
                                f'{clean.geom_type} is not a Polygon!'
                        poly = clean

                    bbox = box(*poly.bounds)
                    g_json = geojson.dumps(mapping(bbox), sort_keys=True)

                    cursor.execute(dml, (rec['properties']['SUBDIV_NM'], 
                                         rec['properties']['TWP'], 
                                         rec['properties']['TNS'],
                                         rec['properties']['RGE'],
                                         rec['properties']['REW'],
                                         rec['properties']['SEC'],
                                         rec['properties']['QTR_TWP'],
                                         rec['properties']['ALLOTMENT'],
                                         rec['properties']['TRACT'],
                                         rec['properties']['LOT'],
                                         rec['properties']['DIVISION'],
                                         rec['properties']['FRACTION'],
                                         rec['properties']['COUNTY'],
                                         rec['properties']['TOWNSHIP'],
                                         rec['properties']['SURVEY_TYP'],
                                         rec['properties']['ObjectID'],
                                         rec['properties']['VMSLOT'],
                                         rec['properties']['OTHER_SUB'],
                                         rec['properties']['Shape_Length'],
                                         rec['properties']['Shape_Area'],
                                         ts.state,  # TMP
                                         g_json,
                                         poly.wkt))
                    if i % 5000 == 0:
                        print(f"{round(i/total*100, 2)}%: {i} of {total}: {rec['properties']['TOWNSHIP']}")

                db.commit()

                print(f"Completed {ts.layer} load.")
        
        cursor.close()

except Exception as e:
    db.rollback()
    raise e

Starting load of Ohio_Sections: 73371 sections...
("Schema: {'properties': OrderedDict([('SUBDIV_NM', 'str:55'), ('TWP', 'int'), "
 "('TNS', 'str:1'), ('RGE', 'int'), ('REW', 'str:1'), ('SEC', 'int'), "
 "('QTR_TWP', 'str:2'), ('ALLOTMENT', 'str:55'), ('TRACT', 'str:35'), ('LOT', "
 "'str:24'), ('DIVISION', 'str:24'), ('FRACTION', 'int'), ('COUNTY', "
 "'str:11'), ('TOWNSHIP', 'str:50'), ('SURVEY_TYP', 'str:50'), ('Shape_Leng', "
 "'float'), ('ObjectID', 'int'), ('VMSLOT', 'str:50'), ('OTHER_SUB', "
 "'str:65'), ('Shape_Length', 'float'), ('Shape_Area', 'float')]), 'geometry': "
 "'MultiPolygon'}")
0.0%: 0 of 73371: CHESTER
6.81%: 5000 of 73371: HILLIAR
13.63%: 10000 of 73371: ETNA
20.44%: 15000 of 73371: BELPRE
27.26%: 20000 of 73371: TRUMBULL
34.07%: 25000 of 73371: BLOOMFIELD
40.89%: 30000 of 73371: BATH
47.7%: 35000 of 73371: MUSKINGUM
Cleaning HARRISON...
54.52%: 40000 of 73371: MILAN
61.33%: 45000 of 73371: FITCHVILLE
68.15%: 50000 of 73371: PLEASANT
74.96%: 55000 of 73371: TRENT